# nuScenes-devkit チュートリアル

nuScenesのチュートリアルになります。

### 前提
[nuScenesサイト](https://www.nuscenes.org/nuscenes#download)からFull Dataset(v1.0)の Miniをダウンロードして中身を/data/sets/nuscenesに格納しておくこと

## nuScenesデータセットとは

Motional（以前のNutonomy）でチームが開発した自律運転のための公開大規模なデータセットです。 Motionalは、無人車両を安全で信頼性が高く、アクセスしやすい現実にしています。データのサブセットを一般に公開することにより、Motionalは、コンピュータービジョンと自律運転に関する公共研究をサポートすることを目指しています。

nuScenesデータセットはボストンとシンガポールで1000の運転シーン(1シーン20秒)を収集しています。

物体検出や追跡といったタスクに向けて、23のオブジェクトクラスにデータセット全体で2Hz周期で3D境界ボックスをアノテーションされています。

nuScenesは6つのカメラ、1つのLidar、5つのレーダー、GPS、IMUからデータを提供する大規模なデータセットとなります。

nuScenesセンサーレイアウトは公開されています。
![](images/car_setup.png)

## nuScenes データセットの簡単な紹介


チュートリアルのこの部分では、データベースをトップダウンで紹介します。データセットは、次の基本的な要素を持つjsonファイルで構成されています。

1. `log` - データが抽出されたログ情報。
2. `scene` - 車の走行シーンの 20 秒の断片。
3. `sample` - 特定のタイムスタンプにおけるシーンの注釈付きスナップショット。
4. `sample_data` - 特定のセンサー(camera, lidar, radar)から収集されたデータ。
5. `ego_pose` - 特定のタイムスタンプにおける自車両のポーズ。
6. `sensor` - 特定のセンサータイプ。
7. `calibrated sensors` - 特定の車両で調整された特定のセンサーの定義。
8. `instance` - 観察したすべてのオブジェクト インスタンスの列挙。
9. `category` - オブジェクト カテゴリの分類 (例: 車両、人間)。
10. `attribute` - カテゴリは同じままで変更できるインスタンスのプロパティ。
11. `visibility` - 6 台の異なるカメラから収集されたすべての画像で表示されるピクセルの割合。
12. `sample_annotation` - 対象となるオブジェクトの注釈付きインスタンス。
13. `map` - トップダウン ビューからバイナリ セマンティック マスクとして保存されるマップ データ。

nuScenesスキーマは公式サイトで視覚化されています。詳細については[nuScenes schema](https://github.com/nutonomy/nuscenes-devkit/blob/master/docs/schema_nuscenes.md) ページを参照してください。
![](https://www.nuscenes.org/public/images/nuscenes-schema.svg)

## 環境構築(jupyter, Google Colab)

<br>
<a href="https://colab.research.google.com/github/nutonomy/nuscenes-devkit/blob/master/python-sdk/tutorials/nuscenes_tutorial.ipynb">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" align="left">
</a>
<br>

ノートブックで実行している場合は、下のセルのコメントを解除して環境構築およびnuScenes-miniデータセットの取得を実行できます。

In [ ]:
# !mkdir -p /data/sets/nuscenes  # Make the directory to store the nuScenes dataset in.

# !wget https://www.nuscenes.org/data/v1.0-mini.tgz  # Download the nuScenes mini split.

# !tar -xf v1.0-mini.tgz -C /data/sets/nuscenes  # Uncompress the nuScenes mini split.

# !pip install nuscenes-devkit &> /dev/null  # Install nuScenes.

## 初期化

In [ ]:
%matplotlib inline
from nuscenes.nuscenes import NuScenes

nusc = NuScenes(version='v1.0-mini', dataroot='/data/sets/nuscenes', verbose=True)

## データセットの概要

### 1. `scene`

nuScenes は、それぞれ約20秒の ***1000シーン*** にわたるアノテーション付きサンプルを備えた大規模なデータベースです。読み込まれたデータベースにあるシーンを見てみましょう。

In [ ]:
nusc.list_scenes()

シーンのメタデータを見てみましょう

In [ ]:
my_scene = nusc.scene[0]
my_scene

### 2. `sample`

シーンでは、0.5 秒 (2 Hz) ごとにデータにアノテーションを付けます。

`sample` を、***特定のタイムスタンプでのシーンの注釈付きキーフレーム*** として定義します。キーフレームとは、すべてのセンサーからのデータのタイムスタンプが、それが指すサンプルのタイムスタンプに非常に近いフレームです。

では、このシーンの最初の注釈付きサンプルを見てみましょう。

In [ ]:
first_sample_token = my_scene['first_sample_token']

# The rendering command below is commented out because it tends to crash in notebooks
# nusc.render_sample(first_sample_token)

メタデータを調べてみましょう

In [ ]:
my_sample = nusc.get('sample', first_sample_token)
my_sample

便利なメソッドは `list_sample()` です。これは、`sample` に関連付けられたすべての関連する `sample_data` キーフレームと `sample_annotation` を一覧表示します。これについては、後続の部分で詳しく説明します。

In [ ]:
nusc.list_sample(my_sample['token'])

### 3. `sample_data`

nuScenes データセットには、完全なセンサー スイートから収集されたデータが含まれています。したがって、シーンの各スナップショットに対して、これらのセンサーから収集されたデータ ファミリへの参照を提供します。

これらにアクセスするための `data` キーを提供します。

In [ ]:
my_sample['data']

キーは、センサー スイートを構成するさまざまなセンサーを参照していることに注意してください。`CAM_FRONT` から取得した `sample_data` のメタデータを見てみましょう。

In [ ]:
sensor = 'CAM_FRONT'
cam_front_data = nusc.get('sample_data', my_sample['data'][sensor])
cam_front_data

特定のセンサーで `sample_data` をレンダリングすることもできます。

In [ ]:
nusc.render_sample_data(cam_front_data['token'])

### 4. `sample_annotation`

`sample_annotation` は、***サンプル内に表示されるオブジェクトの位置を定義するバウンディングボックス*** を参照します。すべての位置データは、グローバル座標系を基準に与えられます。上記の `sample` の例を調べてみましょう。

In [ ]:
my_annotation_token = my_sample['anns'][18]
my_annotation_metadata =  nusc.get('sample_annotation', my_annotation_token)
my_annotation_metadata

さらに詳しく見るためにアノテーションをレンダリングすることもできます。

In [ ]:
nusc.render_annotation(my_annotation_token)

### 5. `instance`

オブジェクトインスタンスは、AVによって検出または追跡される必要があるインスタンスです（特定の車両、歩行者など）。インスタンスメタデータを調べてみましょう。

In [ ]:
my_instance = nusc.instance[599]
my_instance

通常、特定のシーン内の異なるフレームにわたってインスタンスを追跡します。ただし、異なるシーンにわたってインスタンスを追跡することはありません。この例では、特定のシーンにわたってこのインスタンスの注釈付きサンプルが 16 個あります。

In [ ]:
instance_token = my_instance['token']
nusc.render_instance(instance_token)

インスタンスレコードは最初と最後のアノテーショントークンを記録します。これらをレンダリングしてみましょう。

In [ ]:
print("First annotated sample of this instance:")
nusc.render_annotation(my_instance['first_annotation_token'])

In [ ]:
print("Last annotated sample of this instance")
nusc.render_annotation(my_instance['last_annotation_token'])

### 6. `category`

`カテゴリ` は、注釈のオブジェクト割り当てです。データベースにあるカテゴリ テーブルを見てみましょう。テーブルには、さまざまなオブジェクト カテゴリの分類が含まれており、サブカテゴリ (ピリオドで区切られています) もリストされています。

In [ ]:
nusc.list_categories()

カテゴリ レコードには、特定のカテゴリの名前と説明が含まれます。

In [ ]:
nusc.category[9]

さまざまなカテゴリの定義については、[nuscenes-devkit](https://github.com/nutonomy/nuscenes-devkit) の `instructions_nuscenes.md` を参照してください。

### 7. `attribute`

`属性` は、カテゴリは同じままで、シーンのさまざまな部分で変化する可能性があるインスタンスのプロパティです。ここでは、提供されている属性と、特定の属性に関連付けられている注釈の数をリストします。

In [ ]:
nusc.list_attributes()

1つのシーンで属性がどのように変化するかの例を見てみましょう

In [ ]:
my_instance = nusc.instance[27]
first_token = my_instance['first_annotation_token']
last_token = my_instance['last_annotation_token']
nbr_samples = my_instance['nbr_annotations']
current_token = first_token

i = 0
found_change = False
while current_token != last_token:
    current_ann = nusc.get('sample_annotation', current_token)
    current_attr = nusc.get('attribute', current_ann['attribute_tokens'][0])['name']
    
    if i == 0:
        pass
    elif current_attr != last_attr:
        print("Changed from `{}` to `{}` at timestamp {} out of {} annotated timestamps".format(last_attr, current_attr, i, nbr_samples))
        found_change = True

    next_token = current_ann['next']
    current_token = next_token
    last_attr = current_attr
    i += 1

### 8. `visibility`

`visibility` は、4 つのビンにグループ化された 6 つのカメラ フィード全体に表示される特定の注釈のピクセルの割合として定義されます。

In [ ]:
nusc.visibility

visibilityが80～100%の`sample_annotation`の例を見てみましょう。

In [ ]:
anntoken = 'a7d0722bce164f88adf03ada491ea0ba'
visibility_token = nusc.get('sample_annotation', anntoken)['visibility_token']

print("Visibility: {}".format(nusc.get('visibility', visibility_token)))
nusc.render_annotation(anntoken)

visibilityが0～40%の`sample_annotation`の例を見てみましょう。

In [ ]:
anntoken = '9f450bf6b7454551bbbc9a4c6e74ef2e'
visibility_token = nusc.get('sample_annotation', anntoken)['visibility_token']

print("Visibility: {}".format(nusc.get('visibility', visibility_token)))
nusc.render_annotation(anntoken)

### 9. `sensor`

nuScenesデータセットは、「nuScenesデータセットとは」で説明した通り、以下のセンサースイートから収集されたデータで構成されています:
- 1 x LIDAR、
- 5 x RADAR、
- 6 x カメラ、

In [ ]:
nusc.sensor

すべての `sample_data` には、どの `sensor` からデータが収集されたかの記録があります (「channel」キーに注意してください)

In [ ]:
nusc.sample_data[10]

### 10. `calibrated_sensor`

`calibrated_sensor` は、特定の車両で調整された特定のセンサー (LIDAR/レーダー/カメラ) の定義で構成されます。例を見てみましょう。

In [ ]:
nusc.calibrated_sensor[0]

`translation` および `rotation` パラメータは、自車両のボディ フレームに対して指定されることに注意してください。

### 11. `ego_pose`

`ego_pose` には、グローバル座標系に対する自車両の位置 (`translation` でエンコード) と方向 (`rotation` でエンコード) に関する情報が含まれています。

In [ ]:
nusc.ego_pose[0]

読み込まれたデータベース内の `ego_pose` レコードの数は `sample_data` レコードの数と同じであることに注意してください。これら 2 つのレコードは 1 対 1 の対応を示します。

### 12. `log`

`log` テーブルには、データが抽出されたログ情報が含まれています。`log` レコードは、定義済みのルートに沿った自車の 1 回の走行に対応します。ログの数とログのメタデータを確認してみましょう。

In [ ]:
print("Number of `logs` in our loaded database: {}".format(len(nusc.log)))

In [ ]:
nusc.log[0]

ログが収集された日付や場所など、さまざまな情報が含まれていることに注意してください。また、データが収集されたマップの情報も表示されます。1 つのログに、重複しない複数のシーンを含めることができることに注意してください。

### 13. `map`

マップ情報は、トップダウンビューからバイナリセマンティックマスクとして保存されます。マップの数とマップのメタデータを確認してみましょう。

In [ ]:
print("There are {} maps masks in the loaded dataset".format(len(nusc.map)))

In [ ]:
nusc.map[0]

## nuScenesの基本

少し技術的な話に移ります。

NuScenes クラスには複数のテーブルがあります。各テーブルはレコードのリストで、各レコードは辞書です。たとえば、カテゴリ テーブルの最初のレコードは次の場所に格納されます。

In [ ]:
nusc.category[0]

カテゴリ テーブルはシンプルです。`name` フィールドと `description` フィールドを保持します。また、一意のレコード識別子である `token` フィールドもあります。レコードは辞書なので、トークンには次のようにアクセスできます。

In [ ]:
cat_token = nusc.category[0]['token']
cat_token

DB内の任意のレコードの「トークン」がわかっている場合は、次のようにしてレコードを取得できます。

In [ ]:
nusc.get('category', cat_token)

_お気づきのとおり、同じレコードが取得されました!_

はい、簡単でした。もっと難しいことに挑戦してみましょう。`sample_annotation` テーブルを見てみましょう。

In [ ]:
nusc.sample_annotation[0]

これには `token` フィールドもあります (すべてに存在します)。さらに、[a-z]*\_token 形式のフィールドがいくつかあります (例: instance_token)。これらはデータベース用語では外部キーであり、別のテーブルを指します。
`nusc.get()` を使用すると、これらのいずれかを一定時間で取得できます。たとえば、visibilityのレコードを見てみましょう。

In [ ]:
nusc.get('visibility', nusc.sample_annotation[0]['visibility_token'])

visibility レコードは、注釈が付けられたときにオブジェクトがどの程度可視であったかを示します。

`instance_token` も取得しましょう。

In [ ]:
one_instance = nusc.get('instance', nusc.sample_annotation[0]['instance_token'])
one_instance

これは `instance` テーブルを指します。このテーブルは、各シーンで遭遇したオブジェクト _インスタンス_ を列挙します。このようにして、特定のオブジェクトのすべての注釈を接続できます。
README テーブルをよく見ると、sample_annotation テーブルがインスタンス テーブルを指していることがわかりますが、インスタンス テーブルには、それを指すすべての注釈がリストされていません。
では、特定のオブジェクト インスタンスのすべての sample_annotations を復元するにはどうすればよいでしょうか。方法は 2 つあります。

1. `nusc.field2token()` を使用します。試してみましょう。

In [ ]:
ann_tokens = nusc.field2token('sample_annotation', 'instance_token', one_instance['token'])

これは、`'instance_token'` == `one_instance['token']` であるすべての sample_annotation レコードのリストを返します。これらをセットに保存しましょう。

In [ ]:
ann_tokens_field2token = set(ann_tokens)

ann_tokens_field2token

`nusc.field2token()` メソッドは汎用的で、同様の状況で使用できます。

2. 特定の状況では、テーブル自体に逆インデックスをいくつか提供します。これはその一例です。

instance レコードには、このインスタンスの最初の注釈を指すフィールド `first_annotation_token` があります。
このレコードの回復は簡単です。

In [ ]:
ann_record = nusc.get('sample_annotation', one_instance['first_annotation_token'])
ann_record

これで、「next」フィールドを使用してこのインスタンスのすべての注釈を走査できるようになりました。試してみましょう。

In [ ]:
ann_tokens_traverse = set()
ann_tokens_traverse.add(ann_record['token'])
while not ann_record['next'] == "":
    ann_record = nusc.get('sample_annotation', ann_record['next'])
    ann_tokens_traverse.add(ann_record['token'])

最後に、nusc.field2token を使用して実行したのと同じ ann_records が回復されたことを確認します。

In [ ]:
print(ann_tokens_traverse == ann_tokens_field2token)

## 逆インデックスとショートカット

nuScenes テーブルは正規化されており、各情報は 1 回だけ提供されます。

たとえば、各 `log` レコードには 1 つの `map` レコードがあります。スキーマを見ると、`map` テーブルには `log_token` フィールドがありますが、`log` テーブルには対応する `map_token` フィールドがないことがわかります。しかし、`log` があり、対応する `map` を見つけたい状況はたくさんあります。では、どうすればよいでしょうか。`nusc.field2token()` メソッドをいつでも使用できますが、これは遅くて不便です。そのため、このような状況を含むいくつかの一般的な状況に対して逆マッピングを追加します。

さらに、特定の情報を取得するために複数のテーブルを調べる必要がある状況もあります。

たとえば、`sample_annotation` のカテゴリ名 (例: `human.pedestrian`) を考えてみましょう。カテゴリはインスタンス レベルの定数であるため、`sample_annotation` テーブルにはこの情報は保持されません。代わりに、`sample_annotation` テーブルは `instance` テーブル内のレコードを指します。これは、`category` テーブル内のレコードを指し、最終的に `name` フィールドに必要な情報が格納されます。

注釈のカテゴリ名を知りたいと思うことはよくあるため、NuScenes クラスの初期化中に `sample_annotation` テーブルに `category_name` フィールドを追加します。

このセクションでは、初期化中に `NuScenes` クラスに追加されるショートカットと逆インデックスをリストします。これらはすべて `NuScenes.__make_reverse_index__()` メソッドで作成されます。

### 逆インデックス
デフォルトで 2 つの逆インデックスを追加します。
* `map_token` フィールドが `log` レコードに追加されます。
* `sample` レコードには、そのレコードのすべての `sample_annotations` と `sample_data` キーフレームへのショートカットがあります。詳細については、前のセクションの `nusc.list_sample()` メソッドを参照してください。

### ショートカット

sample_annotation テーブルには「category_name」ショートカットがあります。

_ショートカットを使用_

In [ ]:
catname = nusc.sample_annotation[0]['category_name']

_ショートカットを使用しない_

In [ ]:
ann_rec = nusc.sample_annotation[0]
inst_rec = nusc.get('instance', ann_rec['instance_token'])
cat_rec = nusc.get('category', inst_rec['category_token'])

print(catname == cat_rec['name'])

sample_data テーブルには、「channel」と「sensor_modality」のショートカットがあります。

In [ ]:
# Shortcut
channel = nusc.sample_data[0]['channel']

# No shortcut
sd_rec = nusc.sample_data[0]
cs_record = nusc.get('calibrated_sensor', sd_rec['calibrated_sensor_token'])
sensor_record = nusc.get('sensor', cs_record['sensor_token'])

print(channel == sensor_record['channel'])

## データの可視化

リストとレンダリングのメソッドを提供します。これらは、開発中の便利なメソッドとして、また独自の視覚化メソッドを構築するためのチュートリアルとして意図されています。これらは NuScenesExplorer クラスに実装されており、NuScenes クラス自体を介したショートカットがあります。

### リスト メソッド
使用可能なリスト メソッドは 3 つあります。

1. `list_categories()` は、幅/長さ/高さ（メートル）とアスペクト比のすべてのカテゴリ、カウント、統計を一覧表示します。

In [ ]:
nusc.list_categories()

2. `list_attributes()` はすべての属性とその数を一覧表示します。

In [ ]:
nusc.list_attributes()

3. `list_scenes()` は、ロードされた DB 内のすべてのシーンを一覧表示します。

In [ ]:
nusc.list_scenes()

### 可視化

まず、画像に LiDAR 点群をプロットしてみましょう。LiDAR を使用すると、周囲を 3D で正確にマッピングできます。

In [ ]:
my_sample = nusc.sample[10]
nusc.render_pointcloud_in_image(my_sample['token'], pointsensor_channel='LIDAR_TOP')

前の画像では、色は自車両から各 LIDAR ポイントまでの距離を示しています。LIDAR の強度もレンダリングできます。次の画像では、前方の交通標識は反射率が高く (黄色)、右側の暗い車両は反射率が低く (紫色) なっています。

In [ ]:
nusc.render_pointcloud_in_image(my_sample['token'], pointsensor_channel='LIDAR_TOP', render_intensity=True)

次に、同じ画像のレーダー点群をプロットしてみましょう。レーダーは LIDAR よりも密度が低いですが、範囲がはるかに広くなります。

In [ ]:
nusc.render_pointcloud_in_image(my_sample['token'], pointsensor_channel='RADAR_FRONT')

また、そのサンプルのすべてのサンプルデータにわたるすべての注釈をプロットすることもできます。レーダーの場合、移動するオブジェクトの速度ベクトルもプロットされていることに注意してください。一部の速度ベクトルは外れ値であり、RadarPointCloud.from_file() の設定を使用してフィルタリングできます。

In [ ]:
my_sample = nusc.sample[20]

# The rendering command below is commented out because it may crash in notebooks
# nusc.render_sample(my_sample['token'])

または、特定のセンサーのみをレンダリングしたい場合は、それを指定することができます。

In [ ]:
nusc.render_sample_data(my_sample['data']['CAM_FRONT'])

さらに、複数のsweepsからの点群を集約して、より密度の高い点群を取得することもできます。

In [ ]:
nusc.render_sample_data(my_sample['data']['LIDAR_TOP'], nsweeps=5, underlay_map=True)
nusc.render_sample_data(my_sample['data']['RADAR_FRONT'], nsweeps=5, underlay_map=True)

上記のレーダー プロットでは、2 台の車両からの非常に信頼性の高いレーダー リターンのみが表示されています。これは、ファイル `nuscenes/utils/data_classes.py` で定義されているフィルター設定によるものです。代わりに、すべてのフィルターを無効にしてすべてのリターンをレンダリングする場合は、`disable_filters()` 関数を使用できます。これにより、より密度の高いポイント クラウドが返されますが、背景のオブジェクトからのリターンが多くなります。デフォルト設定に戻すには、`default_filters()` を呼び出すだけです。

In [ ]:
from nuscenes.utils.data_classes import RadarPointCloud
RadarPointCloud.disable_filters()
nusc.render_sample_data(my_sample['data']['RADAR_FRONT'], nsweeps=5, underlay_map=True)
RadarPointCloud.default_filters()

特定の注釈を可視化することもできます。

In [ ]:
nusc.render_annotation(my_sample['anns'][22])

最後に、完全なシーンをビデオとして可視化できます。ここでは 2 つのオプションがあります:
1. nusc.render_scene_channel() は、特定のチャネルのビデオを可視化します。(終了するには ESC キーを押します)
2. nusc.render_scene() は、すべてのカメラ チャネルのビデオを可視化します。

注: これらのメソッドは可視化に OpenCV を使用しますが、IPython Notebooks でうまく動作しない場合があります。問題が発生した場合は、コマンド ラインからこれらの行を実行してください。

シーン 0061 を取得しましょう。これは素晴らしく密度の高いものです。

In [ ]:
my_scene_token = nusc.field2token('scene', 'name', 'scene-0061')[0]

In [ ]:
# The rendering command below is commented out because it may crash in notebooks
# nusc.render_scene_channel(my_scene_token, 'CAM_FRONT')

また、すべてのカメラ チャネルのビデオを可視化するメソッド nusc.render_scene() もあります。
これには高解像度のモニターが必要であり、このノートブックの外部で実行するのが最適です。

In [ ]:
# The rendering command below is commented out because it may crash in notebooks
# nusc.render_scene(my_scene_token)

最後に、特定の場所のすべてのシーンを地図上に視覚化してみましょう。

In [ ]:
nusc.render_egoposes_on_map(log_location='singapore-onenorth')